### Final Project Approach

Create a library of short moves: W, NW, N, NE, E

Create pathfinding alg that operates within box (100 in. long, 90 in wide from net to net) and the five short moves

Obstacle detection + belief map:

In [1]:
# Code adapted from: https://github.com/bitcraze/crazyflie-lib-python/blob/master/examples/autonomousSequence.py

import time
import numpy as np
import cv2
import matplotlib.pyplot as plt

# CrazyFlie imports:

import cflib.crtp
from cflib.crazyflie import Crazyflie
from cflib.crazyflie.log import LogConfig
from cflib.crazyflie.syncCrazyflie import SyncCrazyflie
from cflib.crazyflie.syncLogger import SyncLogger
from cflib.positioning.position_hl_commander import PositionHlCommander

### k means for obstacle detection

In [2]:
from sklearn.cluster import KMeans
# NOTE: This cell breaking? run:
# pip install scikit-learn

def perform_k_means_clustering(coords, k):
    kmeans = KMeans(n_clusters=k, random_state=0, n_init=10).fit(coords)
    cluster_centers = kmeans.cluster_centers_
    labels = kmeans.labels_
    cluster_sizes = np.bincount(labels)
    return cluster_centers, cluster_sizes


### Determine direction of least obstacles

In [3]:

# Note: This does not account for drone's position in space
def which_way(frame_width, centers, sizes):
    # Step 1: Divide the field of view
    section_width = frame_width / 3

    # Step 2: Assign clusters to sections
    sections = {'left': [], 'forward': [], 'right': []}
    for center, size in zip(centers, sizes):
        if center[0] < section_width:
            sections['left'].append(size)
        elif center[0] < 2 * section_width:
            sections['forward'].append(size)
        else:
            sections['right'].append(size)

    # Step 3: Calculate obstacle density
    densities = {direction: sum(sizes) / section_width for direction, sizes in sections.items()}

    # Step 4: Choose direction
    return min(densities, key=densities.get)

In [4]:
group_number = 16

# Possibly try 0, 1, 2 ...
camera = 0

In [5]:
%matplotlib notebook

In [6]:
# states:
SCANNING = 0
MOVING = 1

In [2]:
cap = cv2.VideoCapture(camera)

state = SCANNING

frames = []

fig, ax = plt.subplots()
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    if state == SCANNING:
        frames.append(frame)

        # plot a piece of text that just says "Scanning..."
        ax.text(0.5, 0.5, 'Scanning...', horizontalalignment='center', verticalalignment='center', fontsize=20)
        fig.canvas.draw()

        if len(frames) >= 10:
            # Compute the median frame
            median_frame = np.median(frames, axis=0).astype(dtype=np.uint8)
            frame = median_frame

            # These define the upper and lower HSV for the red obstacles.
            # Note that the red color wraps around 180, so there are two intervals.
            # Tuning of these values will vary depending on the camera.
            lb1 = (145, 35, 75)
            ub1 = (180, 255, 255)
            lb2 = (0, 75, 75)
            ub2 = (20, 255, 255)

            # Perform contour detection on the input frame.
            hsv1 = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
            hsv2 = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

            # Compute mask of red obstacles in either color range.
            mask1 = cv2.inRange(hsv1, lb1, ub1)
            mask2 = cv2.inRange(hsv2, lb2, ub2)
            # Combine the masks.
            mask = cv2.bitwise_or(mask1, mask2)
            mask = mask[frame.shape[0]//2-100:frame.shape[0]//2+0,:] # crop to 200 tall from center

            # count number of 1s in mask
            num_red_pixels = np.count_nonzero(mask)

            # Create a list of tuples that are the height, width of each 1 in the mask
            # print('starting')
            # NOTE: this swaps the j, i coordinates (bc frame is height x width) so we get proper x, y. But note, y is still inverted.
            ones_coords = [[j, i] for i in range(mask.shape[0]) for j in range(mask.shape[1]) if mask[i][j] == 255]
            if len(ones_coords) == 0:
                continue

            # print(len(ones_coords))
            # print('done')
            # Plot the points
            ax.clear()
            ax.set_xlim([0, mask.shape[1]])
            ax.set_ylim([0, mask.shape[0]])
            #invert y axis
            ax.invert_yaxis()
            ax.set_autoscale_on(False)

            ax.scatter(*zip(*ones_coords))
            
            # Plot the k means cluster center as a big red point
            centers, sizes = perform_k_means_clustering(ones_coords, k=4)
            normalized_sizes = 50 + (sizes - min(sizes)) / (max(sizes) - min(sizes)) * 450
            ax.scatter(*zip(*centers), color='red', s=normalized_sizes)

            # Plot the direction
            direction = which_way(mask.shape[1], centers, sizes)
            # Map direction to a numerical value
            direction_map = {'left': 1/6, 'forward': 1/2, 'right': 5/6}
            direction_value = direction_map.get(direction, 0)

            # Plot a green vertical line over the direction on the graph
            ax.axvline(x=direction_value * mask.shape[1], color='green', linewidth=5)

            fig.canvas.draw()

            # Switch to scanning mode
            state = MOVING
            frames = []

    elif state == MOVING:
        ax.text(0.5, 0.5, 'Moving...', horizontalalignment='center', verticalalignment='center', fontsize=20)
        fig.canvas.draw()
        
        # Perform moving operations
        time.sleep(2)
        
        # Switch to scanning mode
        state = SCANNING
        ax.clear()

    # Hit q to quit.
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture
cap.release()
cv2.destroyAllWindows()

NameError: name 'cv2' is not defined